In [9]:
#imports

import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import geojson
from shapely import geometry
from sentinelsat import SentinelAPI
from datetime import date
import utm
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
from ipyleaflet import Polygon as ipylPolygon

In [10]:
def commandline_simulator(switch, place_name, dir_address, date_range, cloudness, uuid = ''):
#date_range = ('AAAAMMDD', 'AAAAMMDD') (ex: ('20210501', '20210531'))
#place_name = (country,city)
#cloudcoverpercentage=(0, cloudness)

    address={'country':place_name[0],'city':place_name[1]}  # my cities
    geolocator = Nominatim(user_agent="Pio")
    location = geolocator.geocode(address) #getting latitude and longitude (geographic coordinates)
    
    #converting to utm projection
    
    x,y, zone, zoneLetter = utm.from_latlon(location.latitude, location.longitude) 
    x = round(x,2)
    y = round(y,2) #getting the first 2 decimals
    (xoffset,yoffset) = (15000,15000)
    
    upl = (x-xoffset, y+yoffset) # upper left
    upr = (x+xoffset, y+yoffset) # upper right
    dwr = (x+xoffset, y-yoffset) # down right
    dwl = (x-xoffset, y-yoffset) # down left
    
    utmCoord = [upl,upr,dwr,dwl]
    
    #converting back to geographic coordinates
    
    geoCoord = []
    for p in utmCoord:
        lat, lon = utm.to_latlon(*p,zone,zoneLetter)
        geoCoord.append([round(lon,7),round(lat,7)])
    
    #creation of the polygon

    geoPolygon = geometry.asPolygon(geoCoord)
    geoPolygon.to_wkt()
    
    #getting the api    
    
    api = SentinelAPI('leo_beuque', 'Rbcekouya12', 'https://scihub.copernicus.eu/dhus')
    products = api.query(geoPolygon,
                     date=date_range,
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, min(2*cloudness,100)))
    # conversion to pd.dataframe
    
    df = pd.DataFrame(products).T
    
    #choosing the uuid with best fitting cloudness
    
    if uuid == '':
        
        best_cloudness_diff = min(abs(df.cloudcoverpercentage-cloudness))
        idx = abs(df.cloudcoverpercentage-cloudness) == best_cloudness_diff #we get the image with cloudcoverpercentage nearest to cloudness
        new_uuid = df.uuid.loc[idx][0]
    else:
        new_uuid = uuid
    
    if switch:
        #download image
        api.download(new_uuid, dir_address)
        return
    
    else:
        #print information 
        print ('title:', df.title.loc[idx][0])
        print ('\nlink:',df.link[idx][0])
        print ('\ncloudcoverpercentage:',df.cloudcoverpercentage[idx][0])
        return
    
    

In [12]:
commandline_simulator(True,('France','Paris'),'downloaded_images',('20210601', '20210630'),15,'')

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

ServerError: HTTP status 500 Internal Server Error: HttpResponseException : command: POST https://most.onda-dias.eu/v3/auth/tokens HTTP/1.1 failed with response: HTTP/1.1 429 Too Many Requests; content: []